In [3]:
! git clone https://github.com/acolite/acolite.git

Cloning into 'acolite'...
remote: Enumerating objects: 13006, done.
remote: Counting objects: 100% (2113/2113), done.
remote: Compressing objects: 100% (337/337), done.
remote: Total 13006 (delta 1956), reused 1837 (delta 1775), pack-reused 10893 (from 3)
Receiving objects: 100% (13006/13006), 711.70 MiB | 44.37 MiB/s, done.
Resolving deltas: 100% (8326/8326), done.


In [4]:
%%capture
! pip install rasterio
! pip install pyresample
! pip install netCDF4


In [6]:
import os
import sys
import rasterio
from rasterio.windows import Window
from rasterio.windows import from_bounds
from rasterio.enums import Resampling
from rasterio.transform import Affine
import shutil
import numpy as np
import tempfile
import shutil
from pyproj import Transformer
import gdown
import dask.dataframe as dd
import glob
from tqdm import tqdm
import cv2



In [7]:
ACOLITE_PATH = "./acolite"
sys.path.append(ACOLITE_PATH)
# Import acolite_run
from acolite.acolite.acolite_run import acolite_run

In [8]:
def get_utm_limit(utm_x, utm_y,utm_crs, box_meters, pixel_size=10):
    """Compute WGS84 limit from UTM center for box_meters x box_meters."""
    #utm_crs = "EPSG:32631"  # UTM 31N
    wgs84_crs = "EPSG:4326"
    utm_to_wgs = Transformer.from_crs(utm_crs, wgs84_crs, always_xy=True)
    
    half_box = box_meters / 2
    box_left = utm_x - half_box
    box_right = utm_x + half_box
    box_bottom = utm_y - half_box
    box_top = utm_y + half_box
    
    lon_min, lat_min = utm_to_wgs.transform(box_left, box_bottom)
    lon_max, lat_max = utm_to_wgs.transform(box_right, box_top)
    limit = [lat_min, lon_min, lat_max, lon_max]
    
    print(f"UTM box: left={box_left:.1f}, bottom={box_bottom:.1f}, right={box_right:.1f}, top={box_top:.1f}")
    print(f"WGS84 limit: {limit}")
    return limit, (box_left, box_bottom, box_right, box_top)

In [9]:

# def crop_to_exact_size(input_file, output_file, target_width=256, target_height=256, annot_utm_x=None, annot_utm_y=None):
#     with rasterio.open(input_file) as src:
#         width, height = src.width, src.height
#         print(f"Input shape: {width}x{height}")
#         res = src.res[0]
#         if annot_utm_x is not None and annot_utm_y is not None:
#             x_offset = int((annot_utm_x - src.bounds.left) / res - target_width / 2)
#             y_offset = int((src.bounds.top - annot_utm_y) / res - target_height / 2)
#         else:
#             x_offset = (width - target_width) // 2
#             y_offset = (height - target_height) // 2
#         x_offset = max(0, min(x_offset, width - target_width))
#         y_offset = max(0, min(y_offset, height - target_height))
#         window = Window(x_offset, y_offset, target_width, target_height)
#         data = src.read(window=window)
#         transform = src.window_transform(window)
#         profile = src.profile
#         profile.update(width=target_width, height=target_height, transform=transform)
#         with rasterio.open(output_file, 'w', **profile) as dst:
#             dst.write(data)
#         print(f"Cropped to: {target_width}x{target_height}")
#         print(f"Output UTM bounds: left={transform[2]:.1f}, bottom={transform[5]-target_height*res:.1f}, "
#               f"right={transform[2]+target_width*res:.1f}, top={transform[5]:.1f}")
#         return transform[2], transform[5]

In [12]:
! ls /kaggle/input/litter-windrows-batch-cala

S2A_MSIL1C_20150814T094006_N0500_R036_T33SXC_20231010T034522
S2A_MSIL1C_20150923T094016_N0500_R036_T33SXC_20231028T021114
S2A_MSIL1C_20151115T095252_N0500_R079_T33SXC_20231021T074400
S2A_MSIL1C_20161126T094332_N0500_R036_T33SXC_20231025T160709
S2A_MSIL1C_20181007T094031_N0500_R036_T33SXC_20230727T233522
S2A_MSIL1C_20181010T095031_N0500_R079_T33SXC_20230729T115020
S2A_MSIL1C_20181017T094011_N0500_R036_T33SXC_20230729T163213
S2A_MSIL1C_20181027T094101_N0500_R036_T33SXC_20230817T053449
S2A_MSIL1C_20181106T094201_N0500_R036_T33SXC_20230727T210154
S2A_MSIL1C_20181109T095221_N0500_R079_T33SXC_20230813T172922
S2A_MSIL1C_20200110T094351_N0500_R036_T33SXC_20230603T093842
S2A_MSIL1C_20200409T094031_N0500_R036_T33SXC_20230426T035222
S2A_MSIL1C_20201228T095421_N0500_R079_T33SXC_20230401T085523


In [13]:
# List the tiles in the dataset
safe_files_dir = '/kaggle/input/litter-windrows-batch-cala'
tiles = os.listdir(safe_files_dir)
tiles = [str(t)+'.SAFE' for t in tiles]
tiles

['S2A_MSIL1C_20201228T095421_N0500_R079_T33SXC_20230401T085523.SAFE',
 'S2A_MSIL1C_20151115T095252_N0500_R079_T33SXC_20231021T074400.SAFE',
 'S2A_MSIL1C_20181027T094101_N0500_R036_T33SXC_20230817T053449.SAFE',
 'S2A_MSIL1C_20200409T094031_N0500_R036_T33SXC_20230426T035222.SAFE',
 'S2A_MSIL1C_20181109T095221_N0500_R079_T33SXC_20230813T172922.SAFE',
 'S2A_MSIL1C_20181017T094011_N0500_R036_T33SXC_20230729T163213.SAFE',
 'S2A_MSIL1C_20150814T094006_N0500_R036_T33SXC_20231010T034522.SAFE',
 'S2A_MSIL1C_20181007T094031_N0500_R036_T33SXC_20230727T233522.SAFE',
 'S2A_MSIL1C_20150923T094016_N0500_R036_T33SXC_20231028T021114.SAFE',
 'S2A_MSIL1C_20161126T094332_N0500_R036_T33SXC_20231025T160709.SAFE',
 'S2A_MSIL1C_20200110T094351_N0500_R036_T33SXC_20230603T093842.SAFE',
 'S2A_MSIL1C_20181010T095031_N0500_R079_T33SXC_20230729T115020.SAFE',
 'S2A_MSIL1C_20181106T094201_N0500_R036_T33SXC_20230727T210154.SAFE']

In [9]:
# def get_utm_zone(longitude):
#     return int((longitude + 180) / 6) + 1

# longitude = 15.65  # Example for Calabria
# utm_zone = get_utm_zone(longitude)
# print(f"UTM Zone: {utm_zone}N")  # Outputs UTM Zone for Northern Hemisphere

# from pyproj import Transformer

# lat, lon = 38.5, 16.5  # Example for Calabria
# utm_zone = get_utm_zone(lon)
# epsg_code = f"EPSG:{32600 + utm_zone}"  # UTM Northern Hemisphere Code

# transformer = Transformer.from_crs("EPSG:4326", epsg_code, always_xy=True)
# utm_x, utm_y = transformer.transform(lon, lat)

# print(f"UTM Coordinates: X={utm_x}, Y={utm_y}, Zone={utm_zone}N")


In [14]:
def extract_crs_and_bounds(safe_file):
    #granule_path = os.path.join(os.path.join(safe_files_dir, tiles[0]), tiles[0]+'.SAFE/GRANULE/')
    granule_path = os.path.join(safe_file, 'GRANULE/')
    #print(granule_path)
    granule_subdirs = os.listdir(granule_path)  
    img_data_path = os.path.join(granule_path, os.path.join(granule_subdirs[0]),'IMG_DATA/')
    jpg2_files = os.listdir(img_data_path)
    #print('jpg2_files')
    #print(jpg2_files)
    B02_files = [f  for f in jpg2_files if f.endswith('B02.jp2')]
    #print('B02_files')
    #print(B02_files)
    band_path = os.path.join(img_data_path, B02_files[0])
    print(band_path)
    with rasterio.open(band_path) as src:
        print(f"CRS: {src.crs}")  # Should be EPSG:32631
        print(f"Bounds: {src.bounds}")  # Exact UTM coordinates
        data = src.read(1)
        print(f"B02 Valid Pixels: {np.sum(data > 0)} / {data.size}")
    return src.crs, src.bounds, src.transform, src.res

In [15]:
def get_all_tiles(files):
    """
    Given the list of the parquet annotation file, builds the list of all the 
    annotated tiles.
    """
    df = None
    # Process 10 files per batch
    batch_size = 10
    tiles = []
    for i in range(0, len(files), batch_size):
        batch_files = files[i:i + batch_size]
        
        # Read batch with Dask (lazy loading)
        ddf = dd.read_parquet(batch_files, engine='pyarrow')
        
        # Compute to pandas (triggers parallel read)
        batch_df = ddf.compute()  # Now contains data from 10 files
        tiles.append(batch_df['s2_product'].unique())
       
        print(f"Batch {i//batch_size + 1}: {len(batch_df)} rows (from {len(batch_files)} files)")
    
    tiles = np.unique(np.hstack(tiles))
    tiles = [t.decode('utf-8') for t in tiles]
    return tiles

In [16]:
def find_jp2_ref_file(safe_file):
    granule_path = os.path.join(safe_file, 'GRANULE/')
    granule_subdirs = os.listdir(granule_path)  
    img_data_path = os.path.join(granule_path, os.path.join(granule_subdirs[0]),'IMG_DATA/')
    jpg2_files = os.listdir(img_data_path)
    B02_files = [f  for f in jpg2_files if f.endswith('B02.jp2')]
    if len(B02_files) == 0:
        raise ValueError(f'No B2 band jpg2 file retrieve in the safe file {safe_file}')
    else :
        return os.path.join(img_data_path, B02_files[0])
        
def check_for_invalid_data(file_path, bbox):
    with rasterio.open(file_path) as src:
        # Read the window
        window = src.window(*bbox)
        data = src.read(1, window=window)
    
    # Sentinel-2 often uses 0 for invalid pixels
    invalid_mask = (data == 0)
    
    # Or sometimes very high values (like 65535 for uint16)
    if data.dtype == np.uint16:
        invalid_mask = invalid_mask | (data == 65535)
    
    has_invalid = invalid_mask.any()
    
    if has_invalid:
        invalid_count = invalid_mask.sum()
        print(f"Found {invalid_count} invalid pixels in the bounding box")
    else:
        print("No invalid pixels found in the bounding box")
    return has_invalid

def generate_tile_regions(safe_file_path):
    """
    Returns regions splitting the tile image
    in the form of (xmin, ymin, xmax, ymax)
    """
    w = 10980
    h = 10980
    stepx = 2700
    stepy = 2700
    sx = 3000
    sy = 3000
    regions = []
    for i in range(4):
        for j in range(4):
            regions.append((i*stepx, j*stepy, min(w, i*stepx + sx), min(h, j*stepy + sy)))
    ref_file = find_jp2_ref_file(safe_file_path)
    filtered_regions = []
   
    for r in regions :
        invalid = check_for_invalid_data(ref_file, r)
        if not invalid:
            filtered_regions.append(r)
    regions = {i : r for i, r in enumerate(filtered_regions)}
    return regions

def assign_patches(tile_regions, patches):
    splitted_regions = {}
    for p in patches: 
        for idr, r in tile_regions.items():
            inside = p[0] >= r[0] and p[1] >= r[1] and p[2] <= r[2] and p[3] <= r[3]
            if inside:
                if idr  not in splitted_regions:
                    splitted_regions[idr] = [p]
                else :
                    splitted_regions[idr].append(p)
                break
    return splitted_regions

In [25]:
def generate_tile_mask(tile_df):
    """
    Generate mask for the whole tile.
    Based on dhia's code.
    Not filtering filaments by box_dims size at the moment
    """
    mask = np.zeros((10980, 10980))
    pixels = 0
    #pb = tqdm(tile_df[tile_df["box_dims"] == 3].iterrows())
    pb = tqdm(tile_df.iterrows())
    for index, row in pb:
        x, y = row["pixel_x"], row["pixel_y"]
        if not np.isnan(x) and not np.isnan(y):
            pixels += 1
            x, y = int(x), int(y)
            mask[x, y] = 1
    return mask, pixels
    
def find_subregions_efficient(binary_image, subregion_size, threshold):
    """
    Finds regions with debris pixels in the whole tile annotation mask.
    Using integral images for large binary images.
    """
    h, w = subregion_size
    img_h, img_w = binary_image.shape
    print(f'input maks shape : {binary_image.shape}')
    # Convert to binary and create integral image
    binary = (binary_image == 1).astype(np.uint8)
    integral = cv2.integral(binary)
    
    subregions = []
    covered = np.zeros_like(binary, dtype=bool)
    
    for y in range(0, img_h - h + 1, h):
        for x in range(0, img_w - w + 1, w):
            #if covered[y:y+h, x:x+w].any():
            #    continue
                
            # Calculate sum using integral image
            total = integral[y+h, x+w] - integral[y, x+w] - integral[y+h, x] + integral[y, x]
            white_fraction = total / (h * w)
            #print(f'white_fraction {white_fraction}')
            if total > threshold:
                subregions.append((x, y,  x + w,  y + h, total))
                covered[y:y+h, x:x+w] = True
                
    return subregions

def minimal_1024_cover(small_regions):
    """
    This function could be used to find larger regions around patches
    for ACOLITE application.
    DSF algorithm could be not completely reliable applied to regions of 2560x2560 m unless
    they are very homogeneous
    """
    # Converti le coordinate in celle della griglia 256×256
    cells = set((x // 1024, y // 1024) for (y, x, _, _,_) in small_regions)
    if not cells:
        return []
    large_regions = set()
    for (i, j) in cells:
        large_i = i * 1024
        large_j = j * 1024
        large_regions.add((large_i, large_j))
    return large_regions


def upload_to_drive(file_path, drive_folder_id):
    """Upload file to Google Drive."""
    output = gdown.upload(file_path, parent_id=drive_folder_id, quiet=False)
    print(f"Uploaded {file_path} to Drive folder {drive_folder_id}")
    os.remove(file_path)  # Clear Kaggle disk


In [18]:

def build_tile_df(target, files):
    """
    Retrieves all annotations for a given target tile.
    """
    df = None
    # Process 10 files per batch
    batch_size = 10
    tg_id_1 = target.split('_')[2]
    tg_id_2 = '_'.join(target.split('_')[4:6])
    for i in range(0, len(files), batch_size):
        batch_files = files[i:i + batch_size]
        
        # Read batch with Dask (lazy loading)
        ddf = dd.read_parquet(batch_files, engine='pyarrow')
        
        # Compute to pandas (triggers parallel read)
        batch_df = ddf.compute()  # Now contains data from 10 files
        batch_df["s2_product"] = batch_df["s2_product"].str.decode('utf-8')
        batch_df = batch_df[batch_df["s2_product"].str.contains(tg_id_1) & batch_df["s2_product"].str.contains(tg_id_2)]
        if not batch_df.empty:
            if df is None:
                df = batch_df.copy()
            else:
                df = pd.concat([df, batch_df], ignore_index=True)
        print(f"Batch {i//batch_size + 1}: {len(batch_df)} rows (from {len(batch_files)} files)")
    return df

In [19]:

# Define settings form map-mapper paper
settings = {
    # Ensure TIFF export
    'l2r_export_geotiff': True,  
    # delete .nc once made to geotiff
    'l1r_delete_netcdf' : True,
    'l2w_delete_netcdf' : True,
    'verbosity': 5,  # Detailed logging
    's2_target_res': 10,  # 10m resolution
    'resampling_method': 'nearest' , # Set to nearest neighbor
    'atmospheric_correction_method': 'dark_spectrum',
    'dsf_exclude_bands' : ['B9', 'B10'],
    # resolves issue with none type see this thread - https://odnature.naturalsciences.be/remsem/acolite-forum/viewtopic.php?t=319
    'geometry_type' : 'grids',
    #masking
    'l2w_mask' : False,
    #sunglint
    'glint_correction' : True,
    'dsf_residual_glint_correction' : True,
    'dsf_residual_glint_correction_method' : 'alternative', # index error occuring with default
    'dsf_residual_glint_wave_range' : [1500,2400],
    'glint_force_band' : None,
    'glint_mask_rhos_wave' : 1600,
    'glint_mask_rhos_threshold' : 0.11,
    'reproject' : False
}


In [21]:
#### Stacking 


def stack_bands(acolite_output_dir, stacked_dir, tile):
    # Mapping wawelength to band names
    wl_to_band  ={
        443 : 'B1',
        492 : 'B2',
        560 : 'B3',
        665 : 'B4',
        704 : 'B5',
        740 : 'B6',
        783 : 'B7',
        833 : 'B8',
        865 : 'B8A',
        1614 : 'B11',
        2202 : 'B12'
    }
    rhos_files = [f for f in os.listdir(acolite_output_dir) if 'rhos' in f and '.tif' in f]
    rhos_files = sorted(rhos_files, key=lambda x: int(x.split('_')[-1][:-4]))
    band_files = [(wl_to_band[int(f.split('_')[-1][:-4])], f) for i,f in enumerate(rhos_files) ]

    # Verify all files exist
    for band, filename in band_files:
        if not os.path.exists(os.path.join(acolite_output_dir, filename)):
            raise FileNotFoundError(f"Missing file: {filename}")

    # Reference band (use B4 for metadata, since all bands are 10m)
    reference_band = 'B4'
    reference_file = os.path.join(acolite_output_dir, [f for b, f in band_files if b == reference_band][0])

    # Open the reference GeoTIFF to get metadata
    with rasterio.open(reference_file) as ref:
        ref_profile = ref.profile
        ref_transform = ref.transform
        ref_crs = ref.crs
        ref_width = ref.width
        ref_height = ref.height
        ref_resolution = ref.res  # Should be (10.0, 10.0)

    # Initialize an array to store all bands
    stacked_data = np.zeros((len(band_files), ref_height, ref_width), dtype=np.float32)
    
    # Read each band (no resampling needed, all are 10m)
    for i, (band, filename) in enumerate(band_files):
        file_path = os.path.join(acolite_output_dir, filename)
        with rasterio.open(file_path) as src:
            # Verify resolution matches
            if src.res != ref_resolution:
                raise ValueError(f"Resolution mismatch in {filename}: expected {ref_resolution}, got {src.res}")
            # Verify dimensions match
            if src.width != ref_width or src.height != ref_height:
                raise ValueError(f"Dimensions mismatch in {filename}: expected {ref_width}x{ref_height}, got {src.width}x{src.height}")
            # Read the band
            data = src.read(1, out_dtype=np.float32)  # Single band, float32
            stacked_data[i] = data

    # Update the profile for the stacked GeoTIFF
    stacked_profile = ref_profile.copy()
    stacked_profile.update({
        'count': len(band_files),  # 11 bands
        'dtype': np.float32,  # For reflectance
        'transform': ref_transform,
        'crs': ref_crs,
        'width': ref_width,
        'height': ref_height,
        'nodata': -999  # Optional: Set nodata value (adjust if ACOLITE uses NaN)
    })
    # Determine patch name similar to MARIDA : S2_DATE_TILE_REGION in folder S2_DATE_TILE
    stacked_tif_fname =  'S2_'+('_'.join(tile.split('_')[2:-3]))+'_stacked.tiff'
    stacked_tif = os.path.join(stacked_dir, stacked_tif_fname)
    # Save the stacked GeoTIFF
    with rasterio.open(stacked_tif, 'w', **stacked_profile) as dst:
        dst.write(stacked_data)
        # Set band descriptions in the usual order
        for i, (band, _) in enumerate(band_files, 1):
            dst.set_band_description(i, band)
    
    print(f"Stacked GeoTIFF saved to: {stacked_tif}")

    # Verify the stacked GeoTIFF
    with rasterio.open(stacked_tif) as stacked:
        print(f"Stacked GeoTIFF:")
        print(f"Bounds: {stacked.bounds}")
        print(f"Dimensions: {stacked.width} columns, {stacked.height} rows")
        print(f"Resolution: {stacked.res}")
        print(f"CRS: {stacked.crs}")
        print(f"Number of bands: {stacked.count}")
        print(f"Band descriptions: {stacked.descriptions}")
    return stacked_tif

In [22]:
######## Cropping


# Paths to the original GeoTIFF, stacked GeoTIFF, and output
def crop_stacked_file(stacked_tif, cropped_tif, utm_box, crs, tile_res):
    
    # Computing metadata for the original patch (before ACOLITE correction)
    
    initial_bounds = utm_box  
    original_width = 256
    original_height = 256
    original_transform = Affine.translation(utm_box[0], utm_box[3]) * Affine.scale(10.0, -10.0)
    original_crs = crs
    original_resolution = tile_res

    # Open the stacked GeoTIFF and crop to initial bounds
    with rasterio.open(stacked_tif) as src:
        # Define a window based on the initial bounds
        window = from_bounds(*initial_bounds, transform=src.transform)
        
        # Read the data, matching original dimensions
        window_data = src.read(
            window=window,
            out_shape=(src.count, original_height, original_width),
            resampling=Resampling.nearest  # Nearest neighbor to avoid interpolation
        )
        
        # Get band descriptions to preserve metadata
        band_descriptions = src.descriptions
        
        # Update the transform to match the original GeoTIFF
        window_transform = original_transform
        
        # Update the profile with new metadata
        profile = src.profile
        profile.update({
            'width': original_width,
            'height': original_height,
            'transform': window_transform,
            'crs': original_crs
        })
        
        # Create a new GeoTIFF with the cropped data
        with rasterio.open(cropped_tif, 'w', **profile) as dst:
            dst.write(window_data)
            # Preserve band descriptions (e.g., B1, B2, ...)
            for i, desc in enumerate(band_descriptions, 1):
                if desc:  # Only set if description exists
                    dst.set_band_description(i, desc)
    
    print(f"Cropped GeoTIFF saved to: {cropped_tif}")

    # Verify the cropped GeoTIFF
    with rasterio.open(cropped_tif) as cropped:
        print(f"Cropped GeoTIFF:")
        print(f"Bounds: {cropped.bounds}")
        print(f"Dimensions: {cropped.width} columns, {cropped.height} rows")
        print(f"Resolution: {cropped.res}")
        print(f"CRS: {cropped.crs}")
        print(f"Number of bands: {cropped.count}")
        print(f"Band descriptions: {cropped.descriptions}")

In [17]:
##### Run if you want to extract all tiles from the LW annotation file
#%%time
#all_tiles = get_all_tiles(annotations_files)

In [20]:
### Setting the directory of the annotations file split and converted to parquet format
parquet_annotations_dir = '/kaggle/input/lw-parquet/kaggle/working'
annotations_files = [os.path.join(parquet_annotations_dir, f) for f in os.listdir(parquet_annotations_dir)]

In [82]:
def process_tile(target, safe_files_dir, annotations_files, output_dir):
    acolite_output_dir = os.path.join(output_dir, 'acolite_output/')
    stacked_dir = os.path.join(output_dir, 'stacked/')
    final_dir = os.path.join(output_dir, 'patches/')
    os.makedirs(acolite_output_dir, exist_ok=True)
    os.makedirs(stacked_dir, exist_ok=True)
    os.makedirs(final_dir, exist_ok=True)
    print(f'building annotations dataframe for tile {target}')
    tile_df = build_tile_df(target, annotations_files)
    mask, debris_pixels_count = generate_tile_mask(tile_df)
    print(f'tile mask shape {mask.shape}')
    if debris_pixels_count == 0:
        print(f'no valid debris pixels in tile {target}')
        return False
    patches = find_subregions_efficient(mask, (256, 256), 0)
    print(f'patches with marine debris : {patches}')
    target_file_path = os.path.join(os.path.join(safe_files_dir, target[:-5]), target)
    crs, bounds, tile_transform, tile_res = extract_crs_and_bounds(target_file_path)       
    print(f'crs : {crs}')
    print(f'bounds : {bounds}')
    res = tile_res[0] #sentinel 2 max resolution
    tile_regions = generate_tile_regions(target_file_path)
    patches_per_region = assign_patches(tile_regions, patches)
    print(f'patches per region {patches_per_region}')
    for i, ppr in patches_per_region.items(): #iterates over regions containing patches
        shutil.rmtree(stacked_dir, ignore_errors=True)
        os.makedirs(stacked_dir, exist_ok=True)
        current_region = tile_regions[i]
        current_center = ((current_region[0] + current_region[2])/2 * res + bounds.left,
        (current_region[1] + current_region[3])/2 * res + bounds.bottom)
        #subregion_centers = [((int(p[1] + p[3])/2) * res + bounds.left, int((p[0] + p[2])/2) * res + bounds.bottom) for p in patches]
        print(f'region center : {current_center}')
        #subregion_size = (256 * res, 256 * res)
        current_region_size = ((current_region[2] - current_region[0]) * res, 
                               (current_region[3] - current_region[1]) * res )
        try:
            # Check disk usage
            print("Disk usage before run:")
            !du -sh /kaggle/input/*
            !df -h /kaggle/tmp /kaggle/working
        
            # Clear output directory
            shutil.rmtree(acolite_output_dir, ignore_errors=True)
            os.makedirs(acolite_output_dir, exist_ok=True)


        # Process each subregion
        #for i, (subregion_utm_x, subregion_utm_y) in enumerate(subregion_centers):
            # Clear output directory
            subregion_utm_x, subregion_utm_y = current_center
            print(f'Cleaning previous acolite output')
            shutil.rmtree(acolite_output_dir, ignore_errors=True)
            os.makedirs(acolite_output_dir, exist_ok=True)
            print(f"\nProcessing subregion {i+1}/{len(tile_regions)} centered at UTM ({subregion_utm_x}, {subregion_utm_y})")
    
            # Compute UTM-based limit
            limit, utm_box = get_utm_limit(subregion_utm_x, subregion_utm_y, crs, current_region_size[0]) #subregion_size[0])
            print(f'setting limit : {limit}')
            settings['limit'] = limit
            print(f"Settings for subregion {i+1}: {settings}")
            output_files = acolite_run(settings=settings, inputfile=target_file_path, output=acolite_output_dir)
            print(f"Generated output files {output_files}")
            print('stacking bands')
            stacked_tiff = stack_bands(acolite_output_dir, stacked_dir, target)
            patch_dir =  os.path.join(final_dir, '_'.join((stacked_tiff.split('/')[-1]).split('_')[:-1]))# '_'.join(stacked_tiff.split('_')[:-1])
            print(f'patch dir {patch_dir}')
            for p in ppr:
                print(f'cropping  patch {p}')
                patch_utm_bound = (p[0] * res + bounds.left, p[1] * res + bounds.bottom,
                                   p[2] * res + bounds.left, p[3] * res + bounds.bottom)
                patch_name_prefix = patch_dir.split('/')[-1]
                patch_name = f'{patch_name_prefix}_{int(patch_utm_bound[0])}_{int(patch_utm_bound[1])}.tiff'
                patch_pathname = os.path.join(patch_dir, patch_name)
                os.makedirs(patch_dir, exist_ok=True)
                print(f'patch pathname {patch_pathname}')
                crop_stacked_file(stacked_tiff, os.path.join(patch_dir, patch_pathname), patch_utm_bound, crs, tile_res)
                patch_mask = mask[p[1] : p[3], p[0] : p[2]]
                print(f'patch mask shape {patch_mask.shape}')
                try:
                    cv2.imwrite(os.path.join(patch_dir, f'{patch_pathname[:-4]}_cl.tiff'), patch_mask)
                except Exception as e:
                    print(f"Error writing mask: {e}")
            # Insert Sanity Check for ACOLITE output ? 


    
            #     # Upload to Drive
            #     upload_to_drive(output_file, drive_folder_id)
            #     upload_to_drive(mask_file, drive_folder_id)
    

            # Final disk usage
            print("\nDisk usage after run:")
            ! du -sh /kaggle/input/*
            !df -h /kaggle/tmp /kaggle/working
      
    
        except Exception as e:
            print(f"Exception captured: {e}")


In [72]:
! ls

acolite  acolite_output  patches


In [81]:
! rm -r ./stacked

In [83]:
process_tile(tiles[5], safe_files_dir, annotations_files, '/kaggle/working' )

building annotations dataframe for tile S2A_MSIL1C_20181017T094011_N0500_R036_T33SXC_20230729T163213.SAFE
Batch 1: 0 rows (from 10 files)
Batch 2: 0 rows (from 10 files)
Batch 3: 0 rows (from 10 files)
Batch 4: 0 rows (from 10 files)
Batch 5: 30756 rows (from 10 files)
Batch 6: 0 rows (from 10 files)
Batch 7: 0 rows (from 10 files)
Batch 8: 0 rows (from 10 files)
Batch 9: 0 rows (from 10 files)
Batch 10: 0 rows (from 10 files)
Batch 11: 0 rows (from 10 files)
Batch 12: 0 rows (from 10 files)
Batch 13: 0 rows (from 10 files)
Batch 14: 0 rows (from 10 files)
Batch 15: 0 rows (from 8 files)


30756it [00:01, 19210.06it/s]


tile mask shape (10980, 10980)
input maks shape : (10980, 10980)
patches with marine debris : [(1024, 7424, 1280, 7680, 42), (512, 9984, 768, 10240, 11), (512, 10240, 768, 10496, 19)]
/kaggle/input/litter-windrows-batch-cala/S2A_MSIL1C_20181017T094011_N0500_R036_T33SXC_20230729T163213/S2A_MSIL1C_20181017T094011_N0500_R036_T33SXC_20230729T163213.SAFE/GRANULE/L1C_T33SXC_A017336_20181017T094338/IMG_DATA/T33SXC_20181017T094011_B02.jp2
CRS: EPSG:32633
Bounds: BoundingBox(left=600000.0, bottom=4190220.0, right=709800.0, top=4300020.0)
B02 Valid Pixels: 120560400 / 120560400
crs : EPSG:32633
bounds : BoundingBox(left=600000.0, bottom=4190220.0, right=709800.0, top=4300020.0)
No invalid pixels found in the bounding box
No invalid pixels found in the bounding box
No invalid pixels found in the bounding box
No invalid pixels found in the bounding box
No invalid pixels found in the bounding box
No invalid pixels found in the bounding box
No invalid pixels found in the bounding box
No invalid pixe

/kaggle/working/./acolite/acolite/acolite/acolite_l2r.py:1001: RuntimeWarning: Mean of empty slice
  cur_sel_par = np.sqrt(np.nanmean(np.square((rhod_f-rhop_f)), axis=2))


Wrote rhos_443 (3000, 3049)
S2A_MSI/B1 took 4.6s (RevLUT)
Wrote rhot_492 (3000, 3049)
Computing surface reflectance 2 492 0.980
Interpolating tiles
Wrote rhos_492 (3000, 3049)
S2A_MSI/B2 took 4.1s (RevLUT)
Wrote rhot_560 (3000, 3049)
Computing surface reflectance 3 560 0.921
Interpolating tiles
Wrote rhos_560 (3000, 3049)
S2A_MSI/B3 took 4.3s (RevLUT)
Wrote rhot_665 (3000, 3049)
Computing surface reflectance 4 665 0.953
Interpolating tiles
Wrote rhos_665 (3000, 3049)
S2A_MSI/B4 took 7.3s (RevLUT)
Wrote rhot_704 (3000, 3049)
Computing surface reflectance 5 704 0.946
Interpolating tiles
Wrote rhos_704 (3000, 3049)
S2A_MSI/B5 took 5.1s (RevLUT)
Wrote rhot_740 (3000, 3049)
Computing surface reflectance 6 740 0.956
Interpolating tiles
Wrote rhos_740 (3000, 3049)
S2A_MSI/B6 took 7.4s (RevLUT)
Wrote rhot_783 (3000, 3049)
Computing surface reflectance 7 783 0.983
Interpolating tiles
Wrote rhos_783 (3000, 3049)
S2A_MSI/B7 took 16.8s (RevLUT)
Wrote rhot_833 (3000, 3049)
Computing surface reflect

In [86]:
! ls /kaggle/working/patches/S2_20181017T094011_N0500

S2_20181017T094011_N0500_605120_4290060._cl.tiff
S2_20181017T094011_N0500_605120_4290060.tiff
S2_20181017T094011_N0500_605120_4292620._cl.tiff
S2_20181017T094011_N0500_605120_4292620.tiff
S2_20181017T094011_N0500_610240_4264460._cl.tiff
S2_20181017T094011_N0500_610240_4264460.tiff
